## Mở đầu

- File notebook này được thực thi trên môi trường **Google Colab**

- Để chạy file vui lòng truy cập đường link sau:

Link google colab : https://colab.research.google.com/drive/1uz6-RNaeWgSYVXbgDxGHC_vDDs7xLGM5?usp=sharing

## INSTALL PYSPARK WITH GOOGLE COLAB

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.1.3/spark-3.1.3-bin-hadoop2.7.tgz
!tar xf spark-3.1.3-bin-hadoop2.7.tgz

In [2]:
!pip install -q findspark

In [3]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 31 kB/s 
     |████████████████████████████████| 198 kB 64.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=b81f0d154a48a450cde775ae245b9732bdf639226a20ae6f2a79bd02fffbd411
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [4]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.3-bin-hadoop2.7"

In [5]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
df = spark.sql("select 'spark' as hello ")
df.show()

+-----+
|hello|
+-----+
|spark|
+-----+



## Import thư viện

In [6]:
import pandas as pd
import pyspark.sql.functions as F
from urllib.request import urlopen
from pyspark import SparkContext
from pyspark.sql.functions import collect_list,array,concat,col, explode, lit
from pyspark.sql import SparkSession, Row
from pyspark.ml import Pipeline
from pyspark.ml.fpm import FPGrowth
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, ClusteringEvaluator
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.clustering import KMeans

## 2.1. Khai thác mẫu phổ biến và luật kết hợp (20%)

### 1. Đọc dữ liệu

- Đọc hai tập tin vào PySpark và sử dụng các phương thức tiền xử lý dữ liệu phù hợp để đưa dữ liệu về một DataFrame gồm hai cột theo đúng thứ tự là mã giao dịch (`order_id`) và danh sách sản phẩm (thể hiện bằng tên, không phải mã sản phẩm)

In [7]:
url='https://raw.githubusercontent.com/nnguyet/Lab04_PySpark/main/data/orders.csv'

pd_df=pd.read_csv(url)
df1 = spark.createDataFrame(pd_df)
df1=df1.select("order_id","product_id")

df1.show(5)

+--------+----------+
|order_id|product_id|
+--------+----------+
|       1|     49302|
|       1|     11109|
|       1|     10246|
|       1|     49683|
|       1|     43633|
+--------+----------+
only showing top 5 rows



In [8]:
url='https://raw.githubusercontent.com/nnguyet/Lab04_PySpark/main/data/products.csv'

pd_df=pd.read_csv(url)
df2 = spark.createDataFrame(pd_df)
df2=df2.select("product_id","product_name")
df2.show(5,False)

+----------+-----------------------------------------------------------------+
|product_id|product_name                                                     |
+----------+-----------------------------------------------------------------+
|1         |Chocolate Sandwich Cookies                                       |
|2         |All-Seasons Salt                                                 |
|3         |Robust Golden Unsweetened Oolong Tea                             |
|4         |Smart Ones Classic Favorites Mini Rigatoni With Vodka Cream Sauce|
|5         |Green Chile Anytime Sauce                                        |
+----------+-----------------------------------------------------------------+
only showing top 5 rows



- Kết hợp bảng

In [9]:
df=df1.join(df2,df1.product_id==df2.product_id).select("order_id","product_name")

# show
df.filter(df.order_id==1).show(5,False)

+--------+---------------------------------------------+
|order_id|product_name                                 |
+--------+---------------------------------------------+
|1       |Organic Celery Hearts                        |
|1       |Bag of Organic Bananas                       |
|1       |Organic Hass Avocado                         |
|1       |Organic 4% Milk Fat Whole Milk Cottage Cheese|
|1       |Bulgarian Yogurt                             |
+--------+---------------------------------------------+
only showing top 5 rows



In [10]:
df = df.groupby("order_id").agg(collect_list("product_name").alias("product_name"))

# show
df.show(5,False)

+--------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|order_id|product_name                                                                                                                                                                   

### 2. Áp dụng giải thuật Khai thác mẫu phổ biến và luật kết hợp

- Áp dụng giải thuật khai thác mẫu phổ biến và luật kết hợp trong gói *pyspark.ml.fpm*. Thử nghiệm với một số bộ giá trị ngưỡng `support` và `confidence`.

- *minSupport* = 0.001, *minConfidence* = 0

In [11]:
fp = FPGrowth(itemsCol="product_name",minSupport=0.001, minConfidence=0)
fpm = fp.fit(df)
fpm.setPredictionCol("newPrediction")
fpm.freqItemsets.show(5)
#Display generated association rules.
fpm.associationRules.show(5)
#Transform examines the input items against all the association rules and summarize the consequents as prediction
fpm.transform(df).show(5)

+--------------------+----+
|               items|freq|
+--------------------+----+
|[Organic Tomato B...| 772|
|[Organic Tomato B...| 175|
|[Organic Tomato B...| 144|
|[Organic Tomato B...| 179|
|[Organic Spinach ...| 475|
+--------------------+----+
only showing top 5 rows

+--------------------+--------------------+-------------------+------------------+--------------------+
|          antecedent|          consequent|         confidence|              lift|             support|
+--------------------+--------------------+-------------------+------------------+--------------------+
|[Broccoli Crown, ...|            [Banana]| 0.3690773067331671|2.5860442347085395|0.001127971404400...|
|   [Sugar Snap Peas]|[Bag of Organic B...| 0.2207001522070015|1.8706619038067482|0.001105107119176276|
|[Organic Red Onio...|[Bag of Organic B...|0.34673366834170855|2.9389262202485296|0.001577635680479...|
|[Organic Red Onio...|[Organic Baby Spi...|0.22780569514237856|3.0550038280801664|0.001036514263503

- *minSupport* = 0.0015, *minConfidence* = 0.0001

In [12]:
fp = FPGrowth(itemsCol="product_name",minSupport=0.0015, minConfidence=0.0001)
fpm = fp.fit(df)
fpm.setPredictionCol("newPrediction")
fpm.freqItemsets.show(5)
#Display generated association rules.
fpm.associationRules.show(5)
#Transform examines the input items against all the association rules and summarize the consequents as prediction
fpm.transform(df).show(5)

+--------------------+----+
|               items|freq|
+--------------------+----+
|[Organic Tomato B...| 772|
|[Organic Spinach ...| 475|
|[Whole Milk Ricot...| 347|
| [Medium Salsa Roja]| 275|
|    [Ground Buffalo]| 231|
+--------------------+----+
only showing top 5 rows

+--------------------+--------------------+-------------------+------------------+--------------------+
|          antecedent|          consequent|         confidence|              lift|             support|
+--------------------+--------------------+-------------------+------------------+--------------------+
|[Organic Red Onio...|[Bag of Organic B...|0.34673366834170855|2.9389262202485296|0.001577635680479...|
|[Organic Zucchini...|[Bag of Organic B...| 0.2694736842105263|2.2840679994560045|0.001951085672476...|
|[Organic Zucchini...|            [Banana]| 0.2326315789473684| 1.629998763329342|0.001684335678192807|
|[Sparkling Water ...|[Sparkling Water ...|0.28431372549019607| 11.10584090736622|0.001547149966846

### 3. Bạn có nhận thấy vấn đề gì về hình thức của các luật được tìm thấy hay không (ta không cần quan tâm ngữ nghĩa của dữ liệu). Nếu có, hãy khắc phục điều này.

Hình thức của các luật được tìm thấy nên ở dạng :
antecedent => consequent

In [ ]:
df=fpm.associationRules

df=df.withColumn("c",lit("=>"))
df = df.withColumn("c", array(df["c"]))

df1 = df.select(concat(col("antecedent"),col("c"),col("consequent")).alias("association rules"))
df1.show(5,False)

+--------------------------------------------------------------------------+
|association rules                                                         |
+--------------------------------------------------------------------------+
|[Organic Red Onion, Organic Strawberries, =>, Bag of Organic Bananas]     |
|[Organic Zucchini, Organic Baby Spinach, =>, Bag of Organic Bananas]      |
|[Organic Zucchini, Organic Baby Spinach, =>, Banana]                      |
|[Sparkling Water Berry, =>, Sparkling Water Grapefruit]                   |
|[Organic Grape Tomatoes, Organic Hass Avocado, =>, Bag of Organic Bananas]|
+--------------------------------------------------------------------------+
only showing top 5 rows



## 2.2. Bài 2 – Phân lớp (20%)

Trong bài tập này, bạn sẽ làm việc trên tập tin mushroom.csv biểu diễn dữ liệu các loài nấm. Dữ
liệu có 8124 mẫu, trong đó mỗi mẫu được thể hiện bằng 22 thuộc tính và phân loại thành “edible” 
(e) hoặc “poisonous” (p).
Bạn cần thực nghiệm các giải thuật phân lớp của MLlib trên dữ liệu được cho. Để hoàn tất yêu cầu 
của câu hỏi, các bước cần thực hiện là

### Thông tin tập dữ liệu

In [13]:
url='https://raw.githubusercontent.com/nnguyet/Lab04_PySpark/main/data/mushrooms.csv'

pd_df=pd.read_csv(url)
df = spark.createDataFrame(pd_df)
df.show(5,False)

+-----+---------+-----------+---------+-------+----+---------------+------------+---------+----------+-----------+----------+------------------------+------------------------+----------------------+----------------------+---------+----------+-----------+---------+-----------------+----------+-------+
|class|cap-shape|cap-surface|cap-color|bruises|odor|gill-attachment|gill-spacing|gill-size|gill-color|stalk-shape|stalk-root|stalk-surface-above-ring|stalk-surface-below-ring|stalk-color-above-ring|stalk-color-below-ring|veil-type|veil-color|ring-number|ring-type|spore-print-color|population|habitat|
+-----+---------+-----------+---------+-------+----+---------------+------------+---------+----------+-----------+----------+------------------------+------------------------+----------------------+----------------------+---------+----------+-----------+---------+-----------------+----------+-------+
|p    |x        |s          |n        |t      |p   |f              |c           |n        |k  

In [14]:
# Kết quả phân loại mushrooms.csv với e - edible , p - poisonous 
df.groupBy('class').count().show()

+-----+-----+
|class|count|
+-----+-----+
|    e| 4208|
|    p| 3916|
+-----+-----+



In [15]:
len(df.columns)

23

In [16]:
df.printSchema()

root
 |-- class: string (nullable = true)
 |-- cap-shape: string (nullable = true)
 |-- cap-surface: string (nullable = true)
 |-- cap-color: string (nullable = true)
 |-- bruises: string (nullable = true)
 |-- odor: string (nullable = true)
 |-- gill-attachment: string (nullable = true)
 |-- gill-spacing: string (nullable = true)
 |-- gill-size: string (nullable = true)
 |-- gill-color: string (nullable = true)
 |-- stalk-shape: string (nullable = true)
 |-- stalk-root: string (nullable = true)
 |-- stalk-surface-above-ring: string (nullable = true)
 |-- stalk-surface-below-ring: string (nullable = true)
 |-- stalk-color-above-ring: string (nullable = true)
 |-- stalk-color-below-ring: string (nullable = true)
 |-- veil-type: string (nullable = true)
 |-- veil-color: string (nullable = true)
 |-- ring-number: string (nullable = true)
 |-- ring-type: string (nullable = true)
 |-- spore-print-color: string (nullable = true)
 |-- population: string (nullable = true)
 |-- habitat: string 

In [17]:
df.columns

['class',
 'cap-shape',
 'cap-surface',
 'cap-color',
 'bruises',
 'odor',
 'gill-attachment',
 'gill-spacing',
 'gill-size',
 'gill-color',
 'stalk-shape',
 'stalk-root',
 'stalk-surface-above-ring',
 'stalk-surface-below-ring',
 'stalk-color-above-ring',
 'stalk-color-below-ring',
 'veil-type',
 'veil-color',
 'ring-number',
 'ring-type',
 'spore-print-color',
 'population',
 'habitat']

### Preprocessing features

In [18]:
categorical_cols= df.schema.names [1:]
categorical_cols

['cap-shape',
 'cap-surface',
 'cap-color',
 'bruises',
 'odor',
 'gill-attachment',
 'gill-spacing',
 'gill-size',
 'gill-color',
 'stalk-shape',
 'stalk-root',
 'stalk-surface-above-ring',
 'stalk-surface-below-ring',
 'stalk-color-above-ring',
 'stalk-color-below-ring',
 'veil-type',
 'veil-color',
 'ring-number',
 'ring-type',
 'spore-print-color',
 'population',
 'habitat']

In [19]:
transformed_data=df

In [20]:
for col in categorical_cols :
  str_indexers = StringIndexer(inputCol=col, outputCol=col +'_idx').fit(transformed_data)
  transformed_data=str_indexers.transform(transformed_data)

In [21]:
for col in categorical_cols :
  onehot_encoders = OneHotEncoder(dropLast=False,inputCol=col +'_idx',outputCol=col +'_onehot').fit(transformed_data)
  transformed_data=onehot_encoders.transform(transformed_data)

In [22]:
label= StringIndexer(inputCol=df.schema.names[0], outputCol='idxlabel').fit(transformed_data)
transformed_data=label.transform(transformed_data)

In [23]:
onehot_cols = [col +'_onehot' for col in categorical_cols]
assembler = VectorAssembler(inputCols=onehot_cols, outputCol="features")
transformed_data=assembler.transform(transformed_data)

### 1. Chia dữ liệu thành tập huấn luyện và tập kiểm thử theo tỉ lệ 80:20

In [24]:
train_data, test_data = transformed_data.randomSplit([0.8, 0.2])

### 2. Xây dựng mô hình decision tree trên tập huấn luyện

In [25]:
model_DecisionTree =DecisionTreeClassifier(labelCol='idxlabel', featuresCol='features').fit(train_data)

### 3. Xây dựng mô hình random forest trên tập huấn luyện

In [26]:
model_RandomForest = RandomForestClassifier(labelCol='idxlabel', featuresCol='features', numTrees=200).fit(train_data)

### 4. Đánh giá hai mô hình trên tập kiểm thử

- Đánh giá mô hình **DecisionTree**

In [27]:
# predict and evaluate with  test data
prediction = model_DecisionTree.transform(test_data)
prediction = prediction.select(['probability', 'prediction', 'idxlabel'])
prediction.show()

+-----------+----------+--------+
|probability|prediction|idxlabel|
+-----------+----------+--------+
|  [1.0,0.0]|       0.0|     0.0|
|  [1.0,0.0]|       0.0|     0.0|
|  [1.0,0.0]|       0.0|     0.0|
|  [1.0,0.0]|       0.0|     0.0|
|  [1.0,0.0]|       0.0|     0.0|
|  [1.0,0.0]|       0.0|     0.0|
|  [1.0,0.0]|       0.0|     0.0|
|  [1.0,0.0]|       0.0|     0.0|
|  [1.0,0.0]|       0.0|     0.0|
|  [1.0,0.0]|       0.0|     0.0|
|  [1.0,0.0]|       0.0|     0.0|
|  [1.0,0.0]|       0.0|     0.0|
|  [1.0,0.0]|       0.0|     0.0|
|  [1.0,0.0]|       0.0|     0.0|
|  [1.0,0.0]|       0.0|     0.0|
|  [1.0,0.0]|       0.0|     0.0|
|  [1.0,0.0]|       0.0|     0.0|
|  [1.0,0.0]|       0.0|     0.0|
|  [1.0,0.0]|       0.0|     0.0|
|  [1.0,0.0]|       0.0|     0.0|
+-----------+----------+--------+
only showing top 20 rows



In [28]:
evaluator = MulticlassClassificationEvaluator(labelCol="idxlabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(prediction)
print("Decision Tree - Test Accuracy = %g" % (accuracy))
print("Decision Tree - Test Error = %g" % (1.0 - accuracy))


Decision Tree - Test Accuracy = 0.9957
Decision Tree - Test Error = 0.00429975


- Đánh giá mô hình **Random forest**

In [29]:
# predict and evaluate with  test data
prediction = model_RandomForest.transform(test_data)
prediction = prediction.select(['probability', 'prediction', 'idxlabel'])
prediction.show()

+--------------------+----------+--------+
|         probability|prediction|idxlabel|
+--------------------+----------+--------+
|[0.91228311177136...|       0.0|     0.0|
|[0.91387985496070...|       0.0|     0.0|
|[0.91658007271075...|       0.0|     0.0|
|[0.90067543555052...|       0.0|     0.0|
|[0.90421728331409...|       0.0|     0.0|
|[0.90216074507455...|       0.0|     0.0|
|[0.90677493006324...|       0.0|     0.0|
|[0.89794906795257...|       0.0|     0.0|
|[0.90366831909564...|       0.0|     0.0|
|[0.90349119786511...|       0.0|     0.0|
|[0.91620913118248...|       0.0|     0.0|
|[0.91742884469095...|       0.0|     0.0|
|[0.90417733823210...|       0.0|     0.0|
|[0.91211831920045...|       0.0|     0.0|
|[0.90595050826874...|       0.0|     0.0|
|[0.91678208308417...|       0.0|     0.0|
|[0.92135084712567...|       0.0|     0.0|
|[0.92294759031500...|       0.0|     0.0|
|[0.91792986450204...|       0.0|     0.0|
|[0.91994349002101...|       0.0|     0.0|
+----------

In [30]:
evaluator = MulticlassClassificationEvaluator(labelCol="idxlabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(prediction)
print("Random Forest - Test Accuracy = %g" % (accuracy))
print("Random Forest - Test Error = %g" % (1.0 - accuracy))


Random Forest - Test Accuracy = 0.992015
Random Forest - Test Error = 0.00798526


### 5. Sử dụng pipeline để thiết lập các bước trên thành một luồng xử lý duy nhất

In [31]:
str_indexers = [StringIndexer(inputCol=col, outputCol=col +'_idx') for col in categorical_cols]
onehot_encoders = [OneHotEncoder(dropLast=False, inputCol=col +'_idx', outputCol=col +'_onehot') for col in categorical_cols]
onehot_cols = [col +'_onehot' for col in categorical_cols]
assembler = VectorAssembler(inputCols=onehot_cols, outputCol="features")

label= StringIndexer(inputCol=df.schema.names[0], outputCol='idxlabel')


#### **Decision tree**

In [32]:
# create dt pipeline
dt=DecisionTreeClassifier(labelCol='idxlabel',featuresCol='features')
dt_pipeline = Pipeline(stages=str_indexers+onehot_encoders+[assembler, label, dt])
train_data, test_data = df.randomSplit([0.8, 0.2])
model_dt = dt_pipeline .fit(train_data)
prediction = model_dt.transform(test_data).cache()
prediction = prediction.select(['probability', 'prediction', 'idxlabel'])

In [33]:
evaluator = MulticlassClassificationEvaluator(labelCol="idxlabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(prediction)
print("Decision Tree - Test Accuracy = %g" % (accuracy))
print("Decision Tree - Test Error = %g" % (1.0 - accuracy))

Decision Tree - Test Accuracy = 0.998749
Decision Tree - Test Error = 0.00125078


#### **Random forest**

In [34]:
# create rt pipeline

rd = RandomForestClassifier(labelCol='idxlabel', featuresCol='features', numTrees=200)
rd_pipeline = Pipeline(stages=str_indexers+onehot_encoders+[assembler, label, rd])
train_data, test_data = df.randomSplit([0.8, 0.2])
model_rd = rd_pipeline .fit(train_data)
prediction = model_rd.transform(test_data).cache()
prediction = prediction.select(['probability', 'prediction', 'idxlabel'])

In [35]:
evaluator = MulticlassClassificationEvaluator(labelCol="idxlabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(prediction)
print("Random forest - Test Accuracy = %g" % (accuracy))
print("Random forest - Test Error = %g" % (1.0 - accuracy))

Random forest - Test Accuracy = 0.990148
Random forest - Test Error = 0.00985222


## 2.3. Bài 3 - Gom cụm (20%)

### 1. Đọc tập tin dữ liệu vào PySpark

- Dữ liệu sử dụng ở phần này là tập dữ liệu *`plants`* về sự phân bố của một số loài thực vật ở khu vực Mỹ và Canada (http://archive.ics.uci.edu/ml/datasets/Plants)

In [36]:
url = 'https://raw.githubusercontent.com/nnguyet/Lab04_PySpark/main/data/plants.data'

file_data = []
for line in urlopen(url):
  val = line.decode('latin').strip().split(',')
  file_data.append((val[0], val[1:]))

plant_df = spark.createDataFrame(file_data, ["name", "place"])
plant_df.show(10)

+--------------------+--------------------+
|                name|               place|
+--------------------+--------------------+
|              abelia|            [fl, nc]|
|abelia x grandiflora|            [fl, nc]|
|         abelmoschus|[ct, dc, fl, hi, ...|
|abelmoschus escul...|[ct, dc, fl, il, ...|
|abelmoschus mosch...|            [hi, pr]|
|               abies|[ak, az, ca, co, ...|
|          abies alba|                [nc]|
|      abies amabilis|[ak, ca, or, wa, bc]|
|      abies balsamea|[ct, in, ia, me, ...|
|abies balsamea va...|[ct, in, ia, me, ...|
+--------------------+--------------------+
only showing top 10 rows



### 2. Tiền xử lý dữ liệu

- Sử dụng One-hot encoding

- Do có vùng phân bố không được liệt kê trong tập tin mô tả dữ liệu, ta lấy danh sách các vùng phân bố từ file data (`plants.data`), không cần sử dụng file mô tả (`stateabbr.txt`)

- Dữ liệu sau khi tiền xử lý là DataFrame gồm:

  - Cột đầu tiên (`name`) thể hiện tên loài thực vật và các cột tiếp theo biểu diễn vùng địa lý.

  - Mỗi dòng thể hiện thông tin phân bố địa lý của một loài thực vật – nếu loài thực vật có tại một vùng địa lý thì ô tương ứng mang giá trị 1, ngược lại mang giá trị 0.

In [37]:
plant_df = plant_df.select(plant_df.name, explode(plant_df.place))
plant_df = plant_df.groupBy('name').pivot('col').agg(lit(1)).na.fill(0).sort('name')
plant_df.show(10)

+--------------------+---+---+---+---+---+---+---+---+---+---+---+-----+---+------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|                name| ab| ak| al| ar| az| bc| ca| co| ct| dc| de|dengl| fl|fraspm| ga| gl| hi| ia| id| il| in| ks| ky| la| lb| ma| mb| md| me| mi| mn| mo| ms| mt| nb| nc| nd| ne| nf| nh| nj| nm| ns| nt| nu| nv| ny| oh| ok| on| or| pa| pe| pr| qc| ri| sc| sd| sk| tn| tx| ut| va| vi| vt| wa| wi| wv| wy| yt|
+--------------------+---+---+---+---+---+---+---+---+---+---+---+-----+---+------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|              abelia|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|    0|  1|

- Dữ liệu sau khi tiền xử lý được lưu vào file `plants.csv`.

In [38]:
plant_df.toPandas().to_csv("plants.csv", encoding='latin1', header=True, index=False)

- Thông tin biểu diễn các cột vùng địa lý ở mỗi dòng gộp thành 1 vector lưu vào cột `features` để phục vụ cho thuật toán gom cụm ở sau.

In [39]:
places = plant_df.schema.names[1:]
assem = VectorAssembler(inputCols=places, outputCol='features')
plant_df = assem.transform(plant_df).select('name', 'features')
plant_df.show(10)

+--------------------+--------------------+
|                name|            features|
+--------------------+--------------------+
|              abelia|(70,[12,35],[1.0,...|
|abelia x grandiflora|(70,[12,35],[1.0,...|
|         abelmoschus|(70,[8,9,12,16,19...|
|abelmoschus escul...|(70,[8,9,12,19,22...|
|abelmoschus mosch...|(70,[16,53],[1.0,...|
|               abies|[1.0,1.0,0.0,0.0,...|
|          abies alba|     (70,[35],[1.0])|
|      abies amabilis|(70,[1,5,6,50,65]...|
|      abies balsamea|(70,[0,8,13,17,20...|
|abies balsamea va...|(70,[0,8,13,17,20...|
+--------------------+--------------------+
only showing top 10 rows



### 3. Thực hiện gom cụm bằng giải thuật K-Means và đánh giá kết quả bằng ClusteringEvaluator

- Thử nghiệm với các giá trị k trong khoảng [2..7]

In [40]:
k_val = [2, 3, 4, 5, 6, 7]
evaluate = []
evaluator = ClusteringEvaluator()
for k in k_val:
  kmean = KMeans().setK(k).setSeed(1)
  model = kmean.fit(plant_df)
  predictions = model.transform(plant_df)
  evaluate.append(evaluator.evaluate(predictions))

In [41]:
for i in range(len(k_val)):
  print(f"k = {k_val[i]} - Evaluate: {evaluate[i]}")

k = 2 - Evaluate: 0.7184588176774374
k = 3 - Evaluate: 0.5933659908688372
k = 4 - Evaluate: 0.20735738024326678
k = 5 - Evaluate: 0.5057972983585592
k = 6 - Evaluate: 0.5025219354750663
k = 7 - Evaluate: 0.13091834356758547


- Kết quả thử nghiệm cho thấy k càng tăng thì kết quả đánh giá càng giảm.

- Đánh giá tốt nhất là với k = 2, cho kết quả 0.718

## Tham khảo
 

*   https://www.youtube.com/watch?v=DFk2pLEDwL0

*   https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.ml.classification.DecisionTreeClassifier.html
*   https://youtu.be/1a7bB1ZcZ3k

*   https://www.youtube.com/watch?v=DdxoVAAdf7c

